# Dashboard Using Yahoo! Finance Data

In this example, we demonstrate how to deploy a simple self-contained dashboard.

We deploy an example with [Yahoo! Finance](https://finance.yahoo.com/) for data, but the same method can be used to deploy any kind of self-contained dashboard. If you're looking to use the dashboard to interact with any kind of machine learning model, check out the [SageMaker Dashboards for ML](https://github.com/awslabs/sagemaker-dashboards-for-ml) example instead.

We first import all of the required Python packages for this notebook.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import boto3
from dotenv import load_dotenv
import os
from pathlib import Path
import sagemaker
import warnings

import utils

In [ ]:
dotenv_filepath = Path('../../../../.env').resolve()
if dotenv_filepath.exists():
    load_dotenv()

## Dashboard Development

In [ ]:
image_name = 'dashboard'

In [ ]:
port = 8501

With dashboard development finished, we should now test a Docker container without the local directory mount. Our existing Docker image has outdated dashboard source code files, so we must rebuild the container with the latest source code files. Our `Dockerfile` copies source code files into the image.

In [ ]:
!(cd dashboard && docker build -t {image_name} . --no-cache)

### Amazon ECR Upload

In [ ]:
AWS_ACCOUNT_ID = os.environ['AWS_ACCOUNT_ID']
AWS_REGION = os.environ['AWS_REGION']
DASHBOARD_ECR_REPOSITORY = os.environ['DASHBOARD_ECR_REPOSITORY']

In [ ]:
!docker tag {image_name} {AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com/{DASHBOARD_ECR_REPOSITORY}:latest
!eval $(aws ecr get-login --no-include-email)
!docker push {AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com/{DASHBOARD_ECR_REPOSITORY}:latest

### Amazon ECS Service Start

In [ ]:
DASHBOARD_ECS_CLUSTER = os.environ['DASHBOARD_ECS_CLUSTER']
DASHBOARD_ECR_SERVICE = os.environ['DASHBOARD_ECR_SERVICE']

In [ ]:
!aws ecs update-service --cluster {DASHBOARD_ECS_CLUSTER} --service {DASHBOARD_ECR_SERVICE} --desired-count 1

In [ ]:
DASHBOARD_URL = os.environ['DASHBOARD_URL']
DASHBOARD_ALB = os.environ['DASHBOARD_ALB']

In [ ]:
if DASHBOARD_URL != DASHBOARD_ALB:
    warnings.warn('\n' + '\n'.join([
        "Add CNAME record on your domain before continuing!",
        "from: {}".format(DASHBOARD_URL),
        "to: {}".format(DASHBOARD_ALB),
        "Otherwise you will see 'An error was encountered with the requested page' with Amazon Cognito."
    ]))
print(f"DASHBOARD_URL: https://{DASHBOARD_URL}")

When using Amazon Cognito authentication, you will be greeted by a sign-in page.

A sample user called `dashboard_user` was created during the AWS CloudFormation Stack creation, and the password will have been sent to the email address you provided during launch. You should look out for an email from no-reply@verificationemail.com with the subject 'New Dashboard Account'.

After typing in the temporary password, you will be prompted to enter a new one. You can use this to access the dashboard on other occasions.

## Clean Up

In [ ]:
!aws ecs update-service --cluster {DASHBOARD_ECS_CLUSTER} --service {DASHBOARD_ECR_SERVICE} --desired-count 0